In [1]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from keras import layers, models
import matplotlib.pyplot as plt
import itertools

In [2]:
def load_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')  # Convertir todas las imágenes a modo RGB
        img = img.resize((256, 256))  # Redimensionar las imágenes a 256x256 píxeles
        img = np.array(img)
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

def load_dataset(data_dir):
    images = []
    labels = []
    label_names = sorted(os.listdir(data_dir))
    label_map = {label_name: idx for idx, label_name in enumerate(label_names)}
    
    for label_name in label_names:
        label_dir = os.path.join(data_dir, label_name)
        if os.path.isdir(label_dir):
            for file_name in os.listdir(label_dir):
                if file_name.endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(label_dir, file_name)
                    img = load_image(image_path)
                    if img is not None:
                        images.append(img)
                        labels.append(label_map[label_name])
                    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, label_map

In [3]:
data_dir = '101_ObjectCategories'
images, labels, label_map = load_dataset(data_dir)

In [4]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, train_size=0.8, stratify=labels)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [6]:
dataset_info = {
    'description': 'Caltech101 dataset manually loaded',
    'features': {
        'image': {'shape': (256, 256, 3), 'dtype': 'uint8'},
        'label': {'num_classes': len(label_map), 'names': list(label_map.keys())}
    },
    'num_examples': len(images),
    'splits': {
        'train': {'num_examples': len(train_images)},
        'test': {'num_examples': len(test_images)}
    }
}

In [7]:
(train_dataset, test_dataset), dataset_info = (train_dataset, test_dataset), dataset_info

In [8]:
num_classes = dataset_info['features']['label']['num_classes']

In [9]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(32)
test_dataset = test_dataset.map(preprocess_image).batch(32)

In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Ruta al archivo de pesos descargado manualmente
local_weights_file = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Cargar el modelo VGG16 con los pesos descargados localmente
base_model = VGG16(weights=local_weights_file, include_top=False, input_shape=(224, 224, 3))

# Crear un nuevo modelo que devuelva los mapas de características
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

# Función para extraer características
def extract_features(dataset):
    features = []
    labels = []
    for images, lbls in dataset:
        feature_maps = model.predict(images)
        features.append(feature_maps)
        labels.append(lbls.numpy())
    return features, labels

# Extraer características para los conjuntos de datos de entrenamiento y prueba
train_features, train_labels = extract_features(train_dataset)
test_features, test_labels = extract_features(test_dataset)


1/1 [==============================] - 1s 1s/step


In [12]:
# Flatten the feature maps to create feature vectors
train_features_flat = np.array([feature.flatten() for batch in train_features for feature in batch])
train_labels_flat = np.array([label for batch in train_labels for label in batch])

# Optionally, save the features and labels for later use
np.save('train_features.npy', train_features_flat)
np.save('train_labels.npy', train_labels_flat)

In [13]:
# Load the saved features and labels
train_features_flat = np.load('train_features.npy')
train_labels_flat = np.load('train_labels.npy')

In [14]:
# Example query: Retrieve the 5 nearest neighbors for a test image
from sklearn.neighbors import NearestNeighbors

# Fit the NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(train_features_flat)

# Extract features from a single query image
query_image, _ = next(iter(test_dataset.take(1)))
query_image = query_image[5]  # Get the image in the batch
query_image = tf.expand_dims(query_image, axis=0)  # Add batch dimension
query_features = model.predict(query_image).flatten().reshape(1, -1)
query_image_display = query_image.numpy()

# Find the nearest neighbors
distances, indices = nn_model.kneighbors(query_features)
print("Indices of nearest neighbors:", indices)
print("Distances to nearest neighbors:", distances)

1/1 [==============================] - 0s 267ms/step
Indices of nearest neighbors: [[4796 5869 2862 2685 1700]]
Distances to nearest neighbors: [[32.47804787 34.81202719 35.21873409 35.35756509 35.48318872]]
